FLOW of the Model

1. We have pdfs
pds have to be stored in a databse or locally
when extraction happens we need to fetch the metadata of the pdf also
and store that as a chunk in a sql database

2. Create a doc in a such a way that each split chunk is stored with the metadata (document name, link, etc, page number, case data)

3. Langchain to handle the metadata of the document that the chunk has been extracted on

4. Then store that in a SQL or MongoDB database to extract the link to the PDF and view that

5.

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

# Necessary pip Installations

!pip install --quiet langchain
!pip install -U  --quiet langchain-community
!pip install --quiet pypdf
!pip install --quiet chromadb
!pip install --quiet lark-parser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting 

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

# anyone of either openai or google-generativeai
!pip install -q -U --quiet google-generativeai
!pip install --upgrade google-generativeai
!pip install --upgrade --quiet  langchain-google-genai

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

%pip install --upgrade --quiet  sentence_transformers
%pip install --upgrade --quiet  langchain sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

!pip install google-api-python-client
!pip install langchain

### **Loading the desired model's API KEY**

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

import os
import google.generativeai as genai
import sys
from langchain_google_genai import ChatGoogleGenerativeAI

sys.path.append('../..')

from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY2')

if api_key is None:
    raise ValueError("Environment variable 'GEMINI_API_KEY2' is not set or is None.")

os.environ['GEMINI_API_KEY'] = api_key
os.environ['GOOGLE_API_KEY'] = api_key

# Configure the API key for the Google Generative AI library
genai.configure(api_key=api_key)

# HUGGING FACE API TOKEN
hf_token = userdata.get('HF_TOKEN')

if hf_token is None:
    raise ValueError("Environment variable 'HF_TOKEN' is not set or is None.")

os.environ['HF_TOKEN'] = hf_token


### **Document Loading load_file()**

In [ ]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader, TextLoader

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def load_file(file_type , path=None):
    if path is None:
        raise ValueError("Path must be provided.")

    print("Loading Document......")

    if file_type == "pdf":
        loader = PyPDFLoader(path)

    elif file_type == "url":
        loader = WebBaseLoader(path)

    elif file_type == "text":
        with open(path, 'r') as file:
            pages = file.readlines()
            return pages

    else:
        raise ValueError("Unsupported file_type. Must be one of: 'pdf', 'url', 'text'. ")

    pages = loader.load()
    return pages

# loader = PyPDFLoader("/content/James F. Kurose, Keith Ross - Computer Networking- A Top Down Approach-Pearson (2020).pdf")
# pages = loader.load()

# loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/37signals-is-you.md")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Splitter Function split_doc()**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

def split_doc(pages , chunk_size: int = 1000, chunk_overlap: int = 4):
    if len(pages) == 0:
        raise ValueError("The 'pages' list is empty. Provide valid document content.")

    try:
        # Create a text splitter instance
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            add_start_index=True,
        )

        # Split the documents
        chunks = text_splitter.split_documents(pages)

        # Check if chunks were created successfully
        if not chunks:
            raise ValueError("No chunks were created. Check the input data.")

        print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
        document = chunks[0]
        print(f"First chunk of split doc: {document.page_content} sourced from {document.metadata}")

        return chunks

    except Exception as e:
        print(f"An error occurred while splitting documents: {e}")
        return []


In [ ]:
%pip install --upgrade --quiet  sentence_transformers
%pip install --upgrade --quiet  langchain sentence_transformers

### **Splitter Function split_doc()**

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

import torch

def embedd_chunks(chunks, save_directory=None):
    # Validate chunks
    if not chunks:
        raise ValueError("The 'chunks' argument cannot be empty.")

    # Validate save_directory
    if save_directory is None:
        raise ValueError("The 'save_directory' must be provided.")

    # if not os.path.isdir(save_directory):
    #     raise ValueError(f"The provided 'save_directory' does not exist: {save_directory}")




    try:
        # Remove old database files if any
        !rm -rf ./save_directory

        # Create embeddings from GoogleGenerativeAIEmbeddings
        # Set device to GPU if available, otherwise use CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")

        # model_name = "sentence-transformers/all-mpnet-base-v2"
        # model_kwargs = {'device': device}
        # encode_kwargs = {'normalize_embeddings': False}

        # embeddings = HuggingFaceInferenceAPIEmbeddings(
        #     api_key=inference_api_key,
        #     model_kwargs=model_kwargs,
        #     model_name="sentence-transformers/all-MiniLM-l6-v2",
        #     encode_kwargs = {'normalize_embeddings': False}
        # )

        embeddings = HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2',
            model_kwargs={'device': device}
            )

        # # Create embeddings from HuggingFaceBgeEmbeddings
        # model_name = "BAAI/bge-small-en"
        # model_kwargs = {"device": device}
        # encode_kwargs = {"normalize_embeddings": True}
        # hf = HuggingFaceBgeEmbeddings(
        #     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
        # )
        # embeddings = hf
        # embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

        # Create Chroma vector store DataBase with vector embeddings of pages
        vectorDB = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            persist_directory=save_directory
        )

        # Print the number of documents in the vector store
        print(f"Number of documents in the vector store: {vectorDB._collection.count()}")

        # Persist vector store to disk
        vectorDB.persist()
        print(f"Saved {len(chunks)} chunks to {save_directory}.")

        return vectorDB

    except FileNotFoundError as e:
        print(f"File not found error: {e}")
    except PermissionError as e:
        print(f"Permission error: {e}")
    except ValueError as e:
        print(f"Value error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

### **Finding out similar embeddings by testing a few qns related to the loaded document**

In [ ]:
# import fitz
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

def cases_loading():
# Get the directory of the script and then access the "pdfs" directory
  pdf_directory = '/content/drive/MyDrive/Bail_Saarathi/datasets/Case_Files_PDF/'
  # Print the directory path
  print(pdf_directory)
  pdf_files = os.listdir(pdf_directory)

  chunks = []
  # Iterate over PDF files in the directory
  for each_file in pdf_files:
      try:
          print("Parsing from the pdf files....", each_file)
          # pdf_doc = fitz.open(pdf_directory + each_file)
          pages = load_file("pdf", pdf_directory + each_file)
          chunks += split_doc(pages, 1000, 4)

          # num_pages = pdf_doc.page_count
          # for page_num in range(num_pages):
          #     print("Reading page....", page_num)
          #     try:
          #         current_page = pdf_doc.load_page(page_num).get_text()

          #         print("Total chunks...", len(chunks))
          #         for each_chunk in range(len(chunks)):
          #             date = extract_date_from_chunk(str(chunks[each_chunk]))
          #             if date:
          #                 dict_temp = {'chunk_id': CTR,
          #                              'chunk_text': str(chunks[each_chunk]),
          #                              'page_number': page_num,
          #                              'document_file_name': each_file,
          #                              'case_date': date}
          #                 cursor.execute(
          #                     "INSERT INTO chunks_fact (chunk_id,chunk_text,page_number,document_file_name,case_date) VALUES (?,?,?,?,?)",
          #                     (CTR, str(chunks[each_chunk]), page_num, each_file, date))
          #                 conn.commit()
          #                 CTR += 1
          #     except Exception as e:
          #         print(f"Exception occurred: {e}")

      except Exception as e:
          print(f"Exception occurred: {e}")

  return chunks

In [ ]:
file_chunks = cases_loading()



/content/drive/MyDrive/Bail_Saarathi/datasets/Case_Files_PDF/
Parsing from the pdf files.... _Minor_Palanivel_vs_Sivakami_Ammal_on_1_April_1925.PDF
Loading Document......
Split 20 documents into 80 chunks.
First chunk of split doc: Madras High Court
(Minor) Palanivel ... vs Sivakami Ammal on 1 April, 1925
Equivalent citations: AIR 1925 Mad 841, 90 Ind Cas 165
Author: V Rao
JUDGMENT Venkatasubba Rao, J.
1. The question that is raised by this appeal is whether in execution of a decree for mesne profits, the
shares of the sons of the judgment-debtor, in the joint family property, are liable to be attached and
sold. It is contended for the sons, that the obligation, recognised by the decree, is in respect of a
debt, which it is not the pious duty of the sons, under the Hindu Law, to discharge.
2. In regard to the application of the rule enunciated in the ancient Hindu Law Texts, the Courts
were confronted, from time to time, with great difficulty. The bare statement of the rule is simple
e

In [ ]:
vectorDB = embedd_chunks(file_chunks, '/content/drive/MyDrive/Bail_Saarathi/vector_database/Case_Files_VectorEmbeddings/')

Using device: cpu


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of documents in the vector store: 8512
Saved 5660 chunks to /content/drive/MyDrive/Bail_Saarathi/vector_database/Case_Files_VectorEmbeddings/.


In [ ]:
# question = "Cases with crime and murder"

def related_pages(question, vectorDB, no_of_pages=3):
    docs = vectorDB.similarity_search(question, k=no_of_pages)
    print("Length of similar related documents fetched: ", len(docs))
    print("Content of the docs is as follows: ")
    for doc in docs:
        print()
        print(doc.page_content)
        print(doc.metadata)

In [ ]:
qn = input("Enter a question: ")
related_pages(qn, vectorDB)

Enter a question: murder and theft
Length of similar related documents fetched:  3
Content of the docs is as follows: 

committing theft in the house of their co-villager, they more often than not, resort to give severe
beatings to the thief, being obsessed by the thought that if he be not taught a lesson today, tomorrow
he may commit burglary in their house. In such a situation, scene of mob is created. The mobsters
after having mobbed such a thief could go to any extent. Sometimes, the passions of the rustic
mobsters at such odd hours run so high that they could go even to the extent to lynch the
perpetrator of the crime. As per Harbhagwan Singh retired DSP (Sic.), the mobsters had given
beatings to the deceased. This evidence explicitly exculpates the appellants.
In the ultimate analysis, it follows that the prosecution has dismally failed to bring home guilt
against the appellants. Sequelly, this appeal is accepted setting aside the impugned judgment/ order
of sentence. The appella

In [ ]:
# FOR ALREADY EXISTING CHROMA DB VECTORS

from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Path where the vector DB is saved
save_directory = "/content/drive/MyDrive/Bail_Saarathi/vector_database/Case_Files_VectorEmbeddings/"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the persisted vector database


# Now you can use vectorDB for similarity search
query = "Crime"
results = vectorDB.similarity_search(query)

# Print the results
for result in results:
    print(result)
    print(result.metadata)
    print(result.metadata['source'])
    # print(result.page_content)


Using device: cpu
page_content='far as the ordinary rut of criminal investigation is concerned. It is the normal day-to-day business of
the police to investigate into charges brought before them and, broadly and generally, they have
nothing to gain, not favours at any rate, by subjecting ordinary criminals to needless harassment.
But the crimes, the criminals and even the complainants can occasionally possess extra-ordinary
features. When the even flow of life becomes turbid, the police can be called upon to inquire into
charges arising out of political antagonism. The powerful processes of criminal law can then be
perverted for achieving extraneous ends. Attendant upon such investigations, when the police are
not free agents within their sphere of duty, is a great amount of inconvenience, harassment and
humiliation. That can even take the form of the parading of a respectable person in handcuffs,
apparently on way to a court of justice. The foul deed is done when an adversary is expos

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII


from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain.prompts import PromptTemplate

def saved_vectorDB_loading(save_directory):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Using device: {device}")
  vectorDB = Chroma(
    persist_directory=save_directory,
    embedding_function= HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2',
            model_kwargs={'device': device}
            )
    )
  return vectorDB

In [ ]:
path_vector_store = "/content/drive/MyDrive/Bail_Saarathi/vector_database/Case_Files_VectorEmbeddings/"
save_vectorDB = saved_vectorDB_loading(path_vector_store)

Using device: cpu


In [ ]:
qn = input("Enter a question: ")
related_pages(qn, save_vectorDB)

Enter a question: murder and rape
Length of similar related documents fetched:  3
Content of the docs is as follows: 
Extraction 1:

(l) The allegation of rape against 5 persons was made who were also present among the mob where
interrogation was made.
(m) At the time of alleged rape, the prosecutrix was conceiving 26 weeks pregnancy. No injury has
occurred to the fetus, neither did she immediately go to the Doctor though as per the deposition, she
was feeling unbearable abdomen pain. She gave birth to a girl child in the month of August, 1992 i.e.
after the period of 3 months from the date of FIR and as per the evidence, the girl child at the time
of recording of evidence was 1 € years old.
R/CR.A/101/1995 JUDGMENT DATED: 07/07/2023
(n) They had opportunity to report to the police officers as they had already taken help to leave the
village of the police officer, who is the witness but they did not deem it fit to disclose the commission
of crime i.e. rape to the police officers and Bh

#### **Similarity Search Algorithm**
#### **Maximum Marginal Retrieval (mmr) Algorithm**
##### 1. Addresses Diversity
##### 2. Manages Compression
##### 3. Handling Specificity


In [ ]:
# Defining metadata information
metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page is from the chapter",
        type="integer",
    ),
]

### **Prompting according to the context/topic/theme of the document**

In [ ]:
# from langchain.prompts import PromptTemplate

# def QA_context_prompting():
#     # Enhanced Prompt Template with assistant and scenario parameters
#     template = """
#     You are acting as {assistant}, specialized in Indian legal cases. In the following scenario, your task is to extract key information and summarize the case report.

#     **Scenario**: {scenario}

#     **Task 1: Extract Metadata**
#     Extract the following metadata from the case file:
#     - **Case Name**: {case_name}
#     - **Date of Judgment**: {date}
#     - **Parties Involved**: {parties}
#     - **Court**: {court}
#     - **Judge(s)**: {judges}

#     **Task 2: Summarize the Case**
#     Provide a concise summary of the judgment report and case details, focusing on:
#     - The final judgment and rationale.
#     - Key legal aspects and evidence presented.
#     - Important legal precedents, if any.

#     Your summary should:
#     - Be less than 200 words.
#     - Use formal and precise language.
#     - Avoid unnecessary details.
#     - If you can't find relevant information, state: "No relevant information found."

#     **Case**: {case_name}
#     **Summary**: {summary}
#     **Points realted to the case**: """

#     # Set up the PromptTemplate
#     QA_CHAIN_PROMPT = PromptTemplate(
#         input_variables=["assistant", "scenario", "case_name", "summary", "date", "parties", "court", "judges"],
#         template=template,
#     )
#     return QA_CHAIN_PROMPT


from langchain.prompts import PromptTemplate

def QA_context_prompting():
    template = """
    You are a legal assistant specialized in Indian legal cases. Your task is to analyze the provided case report, identify the most important key charges, and highlight the context of the case.

    **Metadata**:
    - **Case Name**: Automatically identify the case name from the text.
    - **Date of Judgment**: Automatically identify the date of judgment.
    - **Parties Involved**: Automatically identify the parties involved.
    - **Court**: Automatically identify the court name.
    - **Judge(s)**: Automatically identify the judge(s) involved.

    **Key Charges and Context**:
    - Summarize the key charges or disputes in the case.
    - Highlight the most critical aspects and context of the case.
    - Key legal aspects and evidence presented.
    - Focus on the legal precedents and rationale for the judgment.
    - The final judgment and rationale.

    Your response should:
    - Avoid unnecessary details.
    - Focus only on the critical legal points and context.
    - Present the information concisely and clearly.
    - Use formal and precise language.
    - If you can't find relevant information, state: "No relevant information found."

    **Case Analysis**:
    {context}
    """

    QA_context_prompting = PromptTemplate(
        input_variables=["context"],
        template=template,
    )

    return QA_context_prompting



### **Generating Responses From the LLM**

In [ ]:


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# Defining LLM for Question Answering
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def QA_chain(llm, vectorDB, QA_CHAIN_PROMPT):
  # Run chain
  qa_chain_mr = RetrievalQA.from_chain_type(
      llm,
      retriever=vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 10}),
      return_source_documents=True,
      chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
  )
  return qa_chain_mr


### **Conversational Retriever with Memory Functionality**

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI

# Defining LLM for Question Answering
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def QA_with_memory(llm, vectorDB, qa_prompt_template):

  memory = ConversationBufferMemory(
      memory_key="chat_history",
      return_messages=True
  )

  retriever = vectorDB.as_retriever()

  qa = ConversationalRetrievalChain.from_llm(
      llm,
      retriever=retriever,
      memory=memory,
      combine_docs_chain_kwargs={"prompt": qa_prompt_template}
  )
  print(qa)

  return qa

# qa_memory = QA_with_memory(llm, vectorDB, qa_prompt_template)


### **Question Answering with Conversation Memory History**

In [ ]:
def get_response(question, QA_memory):
  result = QA_memory({"question": question})
  # print(result["result"])
  # result["source_documents"][0]
  return result["answer"]

### **User Interaction with the Langchain powered LLM**

In [ ]:
import time

def clear_memory(qa_memory):
    # qa_memory.memory.clear()
    print("Conversation History cleared...")
    time.sleep(2)
    print("Run the doc_chatbot() block again to restart: ")

def restart_chatbot():
    # Function to restart the chatbot
    print("Restarting chatbot...")
    time.sleep(2)
    # Clear memory before restarting
    clear_memory(qa_memory)
    document_chatbot(qa_memory)

def document_chatbot(qa_memory):
    print("Hello Sir...\n"
          "Ready to explore the verse of questions on the DOC you loaded??\n"
          "Find a chair and grab yourself a cold drink...\n"
          "Lemme answer those damn questions for ya...")

    exit_commands = {"bye", "exit", "quit"}
    restart_commands = {"restart", "start over"}

    while True:
        question = input("Enter a question related to the loaded document (or type 'help' for options): ").strip()

        if question.lower() in exit_commands:
            print("Goodbye! Have a great day!")
            # Clear memory before exiting
            clear_memory(qa_memory)
            break

        elif question.lower() in restart_commands:
            print("Restarting chatbot...")
            time.sleep(2)
            # Restart the chatbot
            restart_chatbot()
            break

        elif question.lower() == 'help':
            print("Available commands:")
            print("'help' - Show this help message")
            print("'bye', 'exit', 'quit' - Exit the chatbot")
            print("'restart', 'start over' - Restart the chatbot")
            print("Any other text - Ask a question about the document")

        else:
            response = get_response(question, qa_memory)
            print("Loading....")
            time.sleep(2)
            print(f"Response: \n {response}")

# **Function Calling and Testing**

In [ ]:
# path = "/content/James F. Kurose, Keith Ross - Computer Networking- A Top Down Approach-Pearson (2020).pdf"
def select_file():
  print("1. pdf")
  print("2. url")
  print("3. text")
  print("4. youtube")
  print("5. markdown")
  file_type = input("Enter the type of document: ")
  if(file_type == "1"):
    file_type = "pdf"
  elif(file_type == "2"):
    file_type = "url"
  elif(file_type == "3"):
    file_type = "text"
  elif(file_type == "4"):
    file_type = "youtube"
  elif(file_type == '5'):
    file_type = "markdown"
  else:
    print("Invalid Input")
    return
  return file_type

file_type = select_file()
path = input("Enter the path of the document: ")

# Loading the content of the document into a variable
pages = load_file(file_type, path)

In [ ]:
# Splitting the document into smaller chunks
chunks = split_doc(pages, 1000, 4)

In [ ]:
# Embedding the chunks as a vector
vectorDB = embedd_chunks(chunks, 'vector_database/DCNTextBookVectors/')

In [ ]:
# Test the fetching of related chunks from the similarity search algorithm
question = input("Enter a question related to the loaded document to test the relevenace of similarity search retrieval efficiency: ")
qns = int(input("No of pages that you would like to fetch: "))

# Checking efficiency of related pages
related_pages(question, vectorDB, no_of_pages=qns)

In [ ]:
# Building a context awared prompt
qa_prompt_template = QA_context_prompting()
print(qa_prompt_template)

# Question Answering Using LLM with Conversational History Memory
from langchain_google_genai import ChatGoogleGenerativeAI
# Defining LLM for Question Answering
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
print(llm)


In [ ]:
qa_memory = QA_with_memory(llm, vectorDB, qa_prompt_template)
print(qa_memory)

question1 = "Explain CRC encoder and decoder."

result = qa_memory({"question": question1})
print(result["answer"])

def get_response(question, QA_memory):
  result = QA_memory({"question": question})
  # print(result["result"])
  # result["source_documents"][0]
  return result["answer"]

### **Run the below block to start the BOT**

In [ ]:
# Question Answering with Memory
qa_memory = QA_with_memory(llm, vectorDB, qa_prompt_template)

# User interaction with the model
document_chatbot(qa_memory)

WHAT MORE TO DO???

ADD METADATA INFO

EXTRACTION OF CHUNKs AND PASSSING THAT TO THE LLM MODEL
ALSO DISPLAY
THE DOCUMNET THE=AT CHUNK HAS BEEN EXTRACTED FROM
PROVIDE THE SUMMARY OF THE CASE AS WELL

ALSO ASK QUESTIONS ON THE EXTRACTED DATA

HOw to extract the case details from the case pdf provided
and pass it to the QUERY the similar case files
and also converse on that CASES

In [ ]:
def extract_date_from_chunk(chunk_text):
    # Use regular expression to find date patterns in the chunk text
    date_pattern = r'\b(\d{1,2}\s[a-zA-Z]+\,\s\d{4})\b'
    matches = re.findall(date_pattern, chunk_text)
    if matches:
        return matches[0]  # Return the first match as the date
    else:
        return None  # Return None if no date is found

In [ ]:



text = "This is a test"

query_result = embeddings.embed_query(text)
query_result[:]

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI

# Defining LLM for Question Answering
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def QA_with_memory(llm, vectorDB, qa_prompt_template):

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    retriever = vectorDB.as_retriever()

    qa = ConversationalRetrievalChain.from_llm(
      llm,
      retriever=retriever,
      memory=memory,
      combine_docs_chain_kwargs={"prompt": qa_prompt_template},
      # document_variable_name="input_text"  # Adjust to match your prompt
    )

    return qa

# Initialize the Prompt Template and QA Chain
qa_prompt_template = QA_context_prompting()
qa_memory = QA_with_memory(llm, vectorDB, qa_prompt_template)


In [ ]:
def get_response(question, QA_memory):
  result = QA_memory({"question": question})
  # print(result["result"])
  # result["source_documents"][0]
  return result["answer"]

In [ ]:
qa_prompt_template = QA_context_prompting()
qa_memory = QA_with_memory(llm, vectorDB, qa_prompt_template)

question1 = "no im talking with respect to a new case which might have crime because of religion"

result = qa_memory({"question": question1})
print(result["answer"])

print(get_response("What is the date of the case?", qa_memory))

## Case Analysis

**Metadata:**

- **Case Name:** No relevant information found.
- **Date of Judgment:** May 28, 2021
- **Parties Involved:** Applicant (defendant) and State (prosecution)
- **Court:** No relevant information found.
- **Judge(s):** No relevant information found.

**Key Charges and Context:**

**Key Charges:** The applicant is charged with sexual intercourse with a minor (prosecutrix).

**Context:**

- The applicant was arrested on February 11, 2021, and is seeking regular bail.
- The prosecution alleges that the applicant committed sexual intercourse with the minor.
- The applicant claims innocence and alleges false implication.
- The FIR was filed two months after the alleged incident.
- The applicant has no criminal antecedents.

**Legal Aspects and Evidence:**

- The prosecution relies on the alleged act of sexual intercourse with a minor as evidence.
- The defense argues for the applicant's innocence and points to the delay in filing the FIR.
- The defense also high

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII


from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain.prompts import PromptTemplate

from langchain.vectorstores import Chroma

def saved_vectorDB_loading(save_directory):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Using device: {device}")
  vectorDB = Chroma(
    persist_directory=save_directory,
    embedding_function= HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2',
            model_kwargs={'device': device}
            )
    )
  return vectorDB

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII


from langchain.prompts import PromptTemplate

def Summarization_prompting():
    template = """
    You are a legal assistant specialized in Indian legal cases. Your task is to analyze the provided case report, identify the most important key charges, and highlight the context of the case.

    **Metadata**:
    - **Case Name**: Automatically identify the case name from the text.
    - **Date of Judgment**: Automatically identify the date of judgment.
    - **Parties Involved**: Automatically identify the parties involved.
    - **Court**: Automatically identify the court name.
    - **Judge(s)**: Automatically identify the judge(s) involved.

    **Key Charges and Context**:
    - Summarize the key charges or disputes in the case.
    - Highlight the most critical aspects and context of the case.
    - Key legal aspects and evidence presented.
    - Focus on the legal precedents and rationale for the judgment.
    - The final judgment and rationale.

    Your response should:
    - Avoid unnecessary details.
    - Focus only on the critical legal points and context.
    - Present the information concisely and clearly.
    - Use formal and precise language.
    - If you can't find relevant information, state: "No relevant information found."

    **Case Analysis**:
    {context}
    """

    Summarization_prompting = PromptTemplate(
        input_variables=["context"],
        template=template,
    )

    return Summarization_prompting

Working on extracting the summary of the case, to use that as an input for similarity relevance search

Mapping of the document part left


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Extraction {i+1}:\n\n" + d.page_content + "\nMetaData : " + d.metadata['source'] for i, d in enumerate(docs)]))

# question = "Cases with crime and murder"

def related_pages(text, vectorDB, no_of_pages=3):
    docs = vectorDB.similarity_search(text, k=no_of_pages)
    print("Length of similar related documents fetched: ", len(docs))
    print("Content of the docs is as follows: ")
    pretty_print_docs(docs)




In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

def extract_file_info_from_similar_text(text, vectorDB):

  # Create a dictionary to store the mapped documents
  mapped_docs = {}

  docs = vectorDB.similarity_search(text, k=15)
  print("Length of similar related documents fetched: ", len(docs))
  print("Content of the docs is as follows: ")

  print()
  pretty_print_docs(docs)
  print()
  for doc in docs:
    source_file = doc.metadata['source'] if 'source' in doc.metadata else None
    # Check if page_content is not None
    if doc.page_content:
      if source_file not in mapped_docs:
        mapped_docs[source_file] = {"content": doc.page_content, "page_number": doc.metadata['page']}
      else:
        mapped_docs[source_file]["content"] += "\n" + doc.page_content


  print(mapped_docs.keys())

  return mapped_docs # Moved return statement outside the for loop

In [ ]:
case_keywords = input("Take this after summarzing the case the bail applicant was charrged after:  ")

matching_case_files = extract_file_info_from_similar_text(case_keywords, vectorDB)

summarizing the input from each case file and displaying

In [ ]:


from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

def summary_file_mapp(case_keywords, path_vector_store):
  # Initialize the LLM
  llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

  vectorDB = saved_vectorDB_loading(path_vector_store)

  mapped_docs = extract_file_info_from_similar_text(case_keywords, vectorDB)
  if not mapped_docs:
    print("No relevant documents found.")
    return

  summaries = {}

  # PromptTemplate
  qa_prompt_template = Summarization_prompting()

  for case_file, content in mapped_docs.items():
    # Generate the summary using LLM models (do this with LLama or a better summarization model)
    summary_input = qa_prompt_template.format(context=content)
    message = [HumanMessage(content=summary_input)]

    result = llm(messages=message)

    # Store the result in the summaries dictionary
    summaries[case_file] = result.content
    print("Case_file:     ", case_file, "\n")
    print("Summary:       ", result.content)

    print()
  return summaries


In [ ]:
case_keywords = input("Take this after summarzing the case the bail applicant was charrged after:  ")
path_vector_store = "/content/drive/MyDrive/Bail_Saarathi/vector_database/IPC_Sections_VectorEmbeddings"
summaries = summary_file_mapp(case_keywords, path_vector_store)

Take this after summarzing the case the bail applicant was charrged after:  rape and murder
Length of similar related documents fetched:  15
Content of the docs is as follows: 

Extraction 1:

(l) The allegation of rape against 5 persons was made who were also present among the mob where
interrogation was made.
(m) At the time of alleged rape, the prosecutrix was conceiving 26 weeks pregnancy. No injury has
occurred to the fetus, neither did she immediately go to the Doctor though as per the deposition, she
was feeling unbearable abdomen pain. She gave birth to a girl child in the month of August, 1992 i.e.
after the period of 3 months from the date of FIR and as per the evidence, the girl child at the time
of recording of evidence was 1 € years old.
R/CR.A/101/1995 JUDGMENT DATED: 07/07/2023
(n) They had opportunity to report to the police officers as they had already taken help to leave the
village of the police officer, who is the witness but they did not deem it fit to disclose the

Identification of Key Points from the uploaded bail application

In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

from langchain.prompts import PromptTemplate

def bail_context_prompting():
    template = """
    You are a legal assistant specialized in Indian legal cases. Your task is to analyze the provided bail application report, identify the most important points the applicant he wants the bail for, and highlight the context of the case.

    **Metadata**:
    - **Case Name**: Automatically identify the case name from the text.
    - **Case Number**: Automatically identify the case number.
    - **Date of Application**: Automatically identify the date of judgment.
    - **Type of Bail**: Automatically identify the date of judgment.
    - **Parties Involved**: Automatically identify the parties involved.
    - **Court**: Automatically identify the court name.
    - **Judge(s)**: Automatically identify the judge(s) involved.
    - **Lawyer Name**: Automatically identify the lawyer(s) involved.


    **Key Reasons/Points for Bail by the applicant**:
    - Summarize the key points the applicant requests the bail for.
    - Identify if he has criminal record. Highlight that.
    - Purpose of the bail
    - Highlight the most critical aspects and context of the case.
    - Check what he says about the involvement in the case and his cooperation with the justice system

    **Reasons to be considered for making a judgement for bail to be granted/ not**:
    - Analyse the points and mention the reasons for granting the bail.
    - Check Criminal Record and verify the criminal history/record ofthat person
    - Study the case and check if the applicant can cause potential threat to evidence and people involved in case
    - Check if the bail is an anticipatory bail/ regular bail/ iterim bail (Interim bail has more chances of approval than Regular)
    - Judge if the person is charged for NON BAILABLE Cases (if yes, decision is left to judges, but provide insights)
    - Instances of the person trying to decieve and fool the system

    Your response should:
    - Avoid unnecessary details.
    - Avoid making any decision, leave the decision to judges.
    - Present the information concisely and clearly.
    - Use formal and precise language.
    - If you can't find relevant information, state: "No relevant information found."

    End with "**Note**: This analysis provides information for the court's consideration. The final decision regarding bail rests solely with the judge, who will weigh all relevant factors and evidence before making a judgment."
    also add the further details arguments, verification needed to be made
    **Case Analysis**:
    {context}
    """

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context"],
        template=template,
    )

    return QA_CHAIN_PROMPT


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

from langchain.prompts import PromptTemplate

def key_phrases_extraction_prompt():
    template = """
    You are assisting in a formal police investigation and legal documentation process. Your task is to analyze the provided case record and extract a list of major key phrases that are critical to the investigation. The key phrases should include significant crimes and other relevant terms from the case record.

    **Instructions**:
    - Identify and extract key phrases that are crucial to understanding the case, including specific crimes (e.g., murder, theft, rape) and relevant context (e.g., dowry, financial fraud, cyber scam).
    - Also include phrases of special scenarios that relate to the case and give description of the investiagtion like (broad day light, shop theft, physicaly damage, mentally challenged man, etc)
    - Do **not** include any extra characters, descriptions, or content other than the key phrases.
    - The output should be a clean Python list of key phrases (as strings).

    **Output Format**:
    - The response should be a Python list where each element is a key phrase (as a string).
    - Ensure there are no additional characters or content in the list other than the key phrases.
    - Use formal and precise language.

    **Content Provided**:
    {context}

    **Output Example**:
    ["wife killed", "dowry", "rape", "murder", "cyber scam", "financial fraud", "theft"]
    """

    KEY_PHRASES_PROMPT = PromptTemplate(
        input_variables=["context"],
        template=template,
    )

    return KEY_PHRASES_PROMPT


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII


def case_keywords_extraction(case_record_text):
  # Initialize the LLM
  llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
  qa_prompt_template = key_phrases_extraction_prompt()

  summary_input = qa_prompt_template.format(context=case_record_text)
  message = [HumanMessage(content=summary_input)]

  result = llm(messages=message)

  crimes_list = []
  print(result.content)

  content_list = result.content

  start_index = content_list.find('[')
  end_index = content_list.rfind(']') + 1
  crimes_list = content_list[start_index:end_index]

  print(result)

  return crimes_list


In [ ]:

#  Case Record Case Title: The State vs. [Defendant's Name]  Case Number: [To Be Assigned]  Date of Incident: 12:00 AM, [Date]  Location: [Shop Name], Bangalore  Charges: Pity Theft, Robbery  Defendant: [Defendant's Name]  Address: [Defendant's Address] Occupation: [Defendant's Occupation] Age: [Defendant's Age] Gender: [Defendant's Gender] Legal Representation: [Lawyer's Name]  Chargesheet 1. Overview of the Incident:  On the night of [Date], at approximately 12:00 AM, the defendant was allegedly involved in a criminal act at [Shop Name], located in Bangalore. The incident involved the following charges:  Pity Theft: Alleged petty theft committed by the defendant. Robbery: The defendant is charged with robbery, which involved using force or intimidation to unlawfully take property from the shop. 2. Description of Charges:  a. Pity Theft:  Details of Theft: The defendant is accused of unlawfully taking small items from the shop without the consent of the shop owner. The items stolen include [List of Stolen Items, if known]. Evidence: Surveillance footage from the shop, witness testimonies, and recovered stolen items (if any). b. Robbery:  Details of Robbery: The defendant is alleged to have forcibly taken property from the shop, threatening or using intimidation against the shopkeeper or any employees present at the time. Evidence: Eyewitness accounts, surveillance footage, and any physical evidence related to the force or intimidation used. 3. Witnesses:  Witness 1: [Name, Address, Contact Information] – Eyewitness to the incident. Witness 2: [Name, Address, Contact Information] – Employee at the shop who observed the defendant’s actions. 4. Evidence:  Surveillance Footage: Video recordings showing the defendant’s actions during the incident. Physical Evidence: Items recovered from the defendant that were stolen from the shop (if applicable). Witness Statements: Statements from individuals who witnessed the crime or were present at the scene. 5. Charges Under Relevant Sections:  Pity Theft: IPC Section 378 (Theft) and any applicable local legal provisions. Robbery: IPC Section 390 (Robbery) and IPC Section 392 (Punishment for Robbery). 6. Investigation Summary:  The investigation was conducted by [Investigation Officer's Name], who gathered evidence, interviewed witnesses, and compiled the case details. The collected evidence supports the charges of petty theft and robbery.  7. Next Steps:  Court Hearing: The case is scheduled for a preliminary hearing on [Date]. Bail Status: The defendant's bail status is [Bail Granted/Denied]. Further Proceedings: The case will proceed to trial where the evidence will be presented, and a verdict will be determined based on the charges and the evidence provided. Case Prepared By: [Investigating Officer's Name] [Designation] [Date]  Approved By: [Senior Officer's Name] [Designation] [Date]  Note: This chargesheet is prepared for the purpose of formal legal proceedings and is subject to further review by the court. The defendant is presumed innocent until proven guilty in a court of law.

In [ ]:
bail_applicant_case_record_text = input("Case RECORD: ")
case_keywords_list = case_keywords_extraction(bail_applicant_case_record_text)

print(case_keywords_list)

Case RECORD: Case Title: The State vs. [Defendant's Name]  Case Number: [To Be Assigned]  Date of Incident: 12:00 AM, [Date]  Location: [Shop Name], Bangalore  Charges: Pity Theft, Robbery  Defendant: [Defendant's Name]  Address: [Defendant's Address] Occupation: [Defendant's Occupation] Age: [Defendant's Age] Gender: [Defendant's Gender] Legal Representation: [Lawyer's Name]  Chargesheet 1. Overview of the Incident:  On the night of [Date], at approximately 12:00 AM, the defendant was allegedly involved in a criminal act at [Shop Name], located in Bangalore. The incident involved the following charges:  Pity Theft: Alleged petty theft committed by the defendant. Robbery: The defendant is charged with robbery, which involved using force or intimidation to unlawfully take property from the shop. 2. Description of Charges:  a. Pity Theft:  Details of Theft: The defendant is accused of unlawfully taking small items from the shop without the consent of the shop owner. The items stolen incl

NameError: name 'HumanMessage' is not defined

# **Ultimate Bail_Reckoner_Summary() Function**

In [ ]:

# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

def Bail_Reckoner_Summary(text, case_number, application_number):
  # Initialize the LLM
  llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

  # PromptTemplate
  qa_prompt_template = bail_context_prompting()

  summary_input = qa_prompt_template.format(context=text)
  message = [HumanMessage(content=summary_input)]

  result = llm(messages=message)

  print(f"Case_file:    {case_number}            Application Number: {application_number}")
  print(f"Highlights:       {result.content}")
  print()
  return result.content



In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

# INNOCENT PERSON ARRESTED FOR WRONG CHARGES
# BAIL APPLICATION  1:    *IN THE [NAME OF COURT]* *Bail Application No. [Application Number]* *In the matter of:* *[Applicant's Name]*, *S/o [Father’s Name]*, *R/o [Address]*, *Applicant* *Versus* *State of [State Name]*, *Represented by [Name of the Prosecutor/Police Station]* *Respondent* --- *APPLICATION FOR REGULAR BAIL* *To,* **The Honorable Judge, [Name of the Court], [Address of the Court]** *Date: [Date]* *Subject: Application for Regular Bail* *Respected Sir/Madam,* I, [Applicant's Name], am the applicant in the above-referenced case. I am filing this application seeking regular bail in connection with the charges leveled against me. I respectfully submit the following grounds in support of my application: 1. *Innocence*: I firmly assert my innocence in the matter. I am committed to proving my innocence and clearing my name. I have no criminal record and have always adhered to the law. 2. *Family Responsibility*: I am the sole earning member of my family. My family depends on me for their sustenance and well-being. My continued incarceration will adversely affect their livelihood and create undue hardship. 3. *Cooperation with Justice System*: I am fully prepared to cooperate with the justice system and adhere to any conditions imposed by the court. I am committed to attending all court hearings and providing any necessary assistance to the investigation. 4. *Assurance of Non-Tampering*: I assure the court that I will not tamper with evidence or influence witnesses. I am committed to upholding the integrity of the judicial process and will abide by all conditions set forth by the court. 5. *No Previous Criminal Record*: I have no previous criminal record and am known to be a law-abiding citizen. This reflects my respect for the law and my commitment to maintaining my integrity. In light of the above, I request the Honorable Court to grant me regular bail. I am ready to provide any security or surety as may be required by the court. I assure the court that I will comply with all conditions of bail and will not abscond or evade the judicial process. I kindly request that the Honorable Court consider my application favorably and grant me bail. *Yours sincerely,* *[Applicant's Name]* *S/o [Father’s Name]* *R/o [Address]* *Contact Number: [Phone Number]* *Email: [Email Address]* *Enclosures:* 1. Copy of the FIR 2. Copy of the charge sheet 3. Proof of address 4. Affidavit regarding financial dependency 5. Any other relevant documents

# CRIMINAL WHO'S TRYING TO MOCK THE SYSTEM AND GET FREE
# BAIL APPLICATION  2:    IN THE [NAME OF COURT] Bail Application No. [Application Number] In the matter of: [Applicant's Name], S/o [Father’s Name], R/o [Address], Applicant Versus State of [State Name], Represented by [Name of the Prosecutor/Police Station] Respondent APPLICATION FOR REGULAR BAIL To, The Honorable Judge, [Name of the Court], [Address of the Court] Date: [Date] Subject: Application for Regular Bail Respected Sir/Madam, I, [Applicant's Name], am the applicant in the above-referenced case. I am seeking regular bail in connection with the charges of rape and murder brought against me. I respectfully submit the following grounds for your consideration: Claim of Innocence: I am steadfast in my claim of innocence concerning the charges filed against me. Despite the serious nature of these charges, I maintain that I am not guilty and am committed to proving my innocence through the legal process. Acknowledgment of Past Incidents: It is acknowledged that there have been previous legal issues in my past. However, I wish to emphasize that these were not related to the current charges and do not reflect the nature of the present allegations. I have learned from past experiences and have been striving to make positive changes in my life. Family Responsibility: I am the sole breadwinner for my family. My continued detention has placed a severe financial burden on them. They rely on me for their daily needs, and my absence is causing them undue hardship. Commitment to Cooperation: I assure the court that I will fully cooperate with all legal proceedings and adhere to any conditions imposed. I am committed to attending all court hearings and assisting with the investigation as required. No Interference Assurance: I guarantee that I will not interfere with any evidence or attempt to influence witnesses. I am committed to upholding the integrity of the legal process and ensuring that the judicial system operates without obstruction. Appeal for Compassion: Given the circumstances, I humbly request the court to consider my application for bail. Granting bail would not only allow me to support my family but also enable me to actively participate in my defense. I respectfully request that the Honorable Court grant me regular bail. I am willing to comply with any conditions set by the court and provide necessary sureties to ensure my presence for all proceedings. Yours sincerely, [Applicant's Name] S/o [Father’s Name] R/o [Address] Contact Number: [Phone Number] Email: [Email Address] Enclosures: Copy of the FIR Copy of the charge sheet Proof of address Affidavit regarding family dependency Any other relevant documents
#Case number, application number

In [ ]:

case_number , application_number = "CASE1234", "APPLN6969"

bail_application_text = input("Application:           ")
bail_summary_var = Bail_Reckoner_Summary(bail_application_text , case_number, application_number)

Application:           IN THE [NAME OF COURT] Bail Application No. [Application Number] In the matter of: [Applicant's Name], S/o [Father’s Name], R/o [Address], Applicant Versus State of [State Name], Represented by [Name of the Prosecutor/Police Station] Respondent APPLICATION FOR REGULAR BAIL To, The Honorable Judge, [Name of the Court], [Address of the Court] Date: [Date] Subject: Application for Regular Bail Respected Sir/Madam, I, [Applicant's Name], am the applicant in the above-referenced case. I am seeking regular bail in connection with the charges of rape and murder brought against me. I respectfully submit the following grounds for your consideration: Claim of Innocence: I am steadfast in my claim of innocence concerning the charges filed against me. Despite the serious nature of these charges, I maintain that I am not guilty and am committed to proving my innocence through the legal process. Acknowledgment of Past Incidents: It is acknowledged that there have been previous

# **Ultimate Previous_Cases_With_Summary_Fetch() Function**

In [ ]:

# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII


from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

def Previous_Cases_With_Summary_Fetch(bail_applicant_case_record_text, path_vector_store, no_of_pgs=5):
  # Initialize the LLM
  llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

  vectorDB = saved_vectorDB_loading(path_vector_store)

  case_keywords = case_keywords_extraction(bail_applicant_case_record_text)

  mapped_docs = extract_file_info_from_similar_text(case_keywords, vectorDB)
  if not mapped_docs:
    print("No relevant documents found.")
    return

  summaries = {}

  # PromptTemplate
  qa_prompt_template = Summarization_prompting()

  counter = 0

  for case_file, metadata in mapped_docs.items():
    if counter == no_of_pgs:
      break
    counter = no_of_pgs
    content = metadata['content']
    page_no = metadata['page_number']
    # Generate the summary using LLM models (do this with LLama or a better summarization model)
    summary_input = qa_prompt_template.format(context=content)
    message = [HumanMessage(content=summary_input)]

    result = llm(messages=message)

    # Store the result in the summaries dictionary
    summaries[case_file] = {"content" : result.content, "page_no": page_no}
    print("Case_file:     ", case_file, "\n")
    print("Summary:       ", result.content)
    print("Page No: ", page_no)
    print()
    counter += 1
  return summaries


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

# CASE RECORD:

# CASE RECORD:          Case Title: The State vs. [Defendant's Name]  Case Number: [To Be Assigned]  Date of Incident: 12:00 AM, [Date]  Location: [Shop Name], Bangalore  Charges: Pity Theft, Robbery  Defendant: [Defendant's Name]  Address: [Defendant's Address] Occupation: [Defendant's Occupation] Age: [Defendant's Age] Gender: [Defendant's Gender] Legal Representation: [Lawyer's Name]  Chargesheet 1. Overview of the Incident:  On the night of [Date], at approximately 12:00 AM, the defendant was allegedly involved in a criminal act at [Shop Name], located in Bangalore. The incident involved the following charges:  Pity Theft: Alleged petty theft committed by the defendant. Robbery: The defendant is charged with robbery, which involved using force or intimidation to unlawfully take property from the shop. 2. Description of Charges:  a. Pity Theft:  Details of Theft: The defendant is accused of unlawfully taking small items from the shop without the consent of the shop owner. The items stolen include [List of Stolen Items, if known]. Evidence: Surveillance footage from the shop, witness testimonies, and recovered stolen items (if any). b. Robbery:  Details of Robbery: The defendant is alleged to have forcibly taken property from the shop, threatening or using intimidation against the shopkeeper or any employees present at the time. Evidence: Eyewitness accounts, surveillance footage, and any physical evidence related to the force or intimidation used. 3. Witnesses:  Witness 1: [Name, Address, Contact Information] – Eyewitness to the incident. Witness 2: [Name, Address, Contact Information] – Employee at the shop who observed the defendant’s actions. 4. Evidence:  Surveillance Footage: Video recordings showing the defendant’s actions during the incident. Physical Evidence: Items recovered from the defendant that were stolen from the shop (if applicable). Witness Statements: Statements from individuals who witnessed the crime or were present at the scene. 5. Charges Under Relevant Sections:  Pity Theft: IPC Section 378 (Theft) and any applicable local legal provisions. Robbery: IPC Section 390 (Robbery) and IPC Section 392 (Punishment for Robbery). 6. Investigation Summary:  The investigation was conducted by [Investigation Officer's Name], who gathered evidence, interviewed witnesses, and compiled the case details. The collected evidence supports the charges of petty theft and robbery.  7. Next Steps:  Court Hearing: The case is scheduled for a preliminary hearing on [Date]. Bail Status: The defendant's bail status is [Bail Granted/Denied]. Further Proceedings: The case will proceed to trial where the evidence will be presented, and a verdict will be determined based on the charges and the evidence provided. Case Prepared By: [Investigating Officer's Name] [Designation] [Date]  Approved By: [Senior Officer's Name] [Designation] [Date]  Note: This chargesheet is prepared for the purpose of formal legal proceedings and is subject to further review by the court. The defendant is presumed innocent until proven guilty in a court of law.


In [ ]:
# IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

path_vector_store = "/content/drive/MyDrive/Bail_Saarathi/vector_database/Case_Files_VectorEmbeddings"

bail_applicant_case_record_text = input("Case RECORD: ")
# Output in the form of KEY VALUE PAIR with Keys as file names, Values as summary
Previous_Cases_Maps = Previous_Cases_With_Summary_Fetch(bail_applicant_case_record_text, path_vector_store, 3)

Case RECORD: Case Title: The State vs. [Defendant's Name]  Case Number: [To Be Assigned]  Date of Incident: 12:00 AM, [Date]  Location: [Shop Name], Bangalore  Charges: Pity Theft, Robbery  Defendant: [Defendant's Name]  Address: [Defendant's Address] Occupation: [Defendant's Occupation] Age: [Defendant's Age] Gender: [Defendant's Gender] Legal Representation: [Lawyer's Name]  Chargesheet 1. Overview of the Incident:  On the night of [Date], at approximately 12:00 AM, the defendant was allegedly involved in a criminal act at [Shop Name], located in Bangalore. The incident involved the following charges:  Pity Theft: Alleged petty theft committed by the defendant. Robbery: The defendant is charged with robbery, which involved using force or intimidation to unlawfully take property from the shop. 2. Description of Charges:  a. Pity Theft:  Details of Theft: The defendant is accused of unlawfully taking small items from the shop without the consent of the shop owner. The items stolen incl

In [ ]:
print(Previous_Cases_Maps)

{'/content/drive/MyDrive/Bail_Saarathi/datasets/Case_Files_PDF/State_vs_Neetu_Anr_on_18_February_2021.PDF': {'content': "## Case Analysis:\n\n**Metadata:**\n\n* **Case Name:** No relevant information found.\n* **Date of Judgment:** No relevant information found.\n* **Parties Involved:** No relevant information found.\n* **Court:** No relevant information found.\n* **Judge(s):** No relevant information found.\n\n**Key Charges and Context:**\n\n* **Charge:** Dishonestly receiving stolen property, punishable under Section 411 of the Indian Penal Code (IPC).\n* **Context:** The prosecution relied on the testimony of police witnesses (PW3, PW-4, and PW-6) to establish the accused's guilt. The defense argued that these witnesses were unreliable due to their interest in the case and the lack of public witnesses.\n* **Key Legal Aspects:** The defense highlighted the absence of public witnesses and the refusal of the accused's relatives to join the investigation, questioning the reliability of 

In [ ]:
import shutil

# Path where the vectorDB is currently saved
current_path_vector_store = "/content/vector_emeddings/IPC_vector_embeddings/"

# Path where you want to copy the vectorDB in Google Drive
google_drive_path = "/content/drive/MyDrive/Bail_Saarathi/vector_database/New_IPC_Sections_VectorEmbeddings/"

# Load the saved vectorDB
save_vectorDB = saved_vectorDB_loading(current_path_vector_store)

# Copy the vectorDB persist folder to Google Drive
shutil.copytree(current_path_vector_store, google_drive_path)

print(f"VectorDB has been copied to {google_drive_path}")
